In [1]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [2]:
#visualise maze:
mazeSize = [4,12]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = 47
            
print(giftState)
print(statePositions)

env = gym.make("CliffWalking-v0")

47
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]


In [45]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 36:
        return [0]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(1)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(3)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(0)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(2)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) != 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

def updateTable_qLearning(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    updated = qTable_1[currentState][direction] + alpha*(reward + (max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    currentState = nextState
    return didConverge, changeInQ

In [46]:
epsilonValue = 0.35
alpha = 0.65
convergenceThresh = 0.01
revistPenalty = -0.25
totalTrials = 0
currentState = 36

In [47]:
def qLearning(maxEpisodes, currentTrial):
    global currentState
    global revisitPenalty
    global totalTrials
    global totalTime_start
    
    firstPath = []
    firstPathEpisode = []
    foundFirstPath = False
    
    currentState = 36
    currentEpisode = 1
    converged = False

    resetTable()
    env.reset()
    
    start_time = time.time()
    
    while currentEpisode <= maxEpisodes:
        if converged:
            break

        direction = nextStep(currentState)
        if not foundFirstPath:
            firstPath.append([currentState, direction])
        nextState, reward, terminated, truncated, info = env.step(direction)

        converged, changeInQ = updateTable_qLearning(direction, nextState, reward)

        if terminated or truncated or converged:
            observation, info = env.reset()
            currentState = 0
            if not converged:
                currentEpisode += 1
            if not foundFirstPath:
                firstPath = []


        if converged:
            end_time = time.time()
        
        time.sleep(0.001)
        clear_output(wait=True)
        if totalTrials != 0:
            print("Trial: " + 'Q-Learning, ' + 'trialType '+ str(trialType) + ', Trial ' + str(currentTrial) + "/" + str(totalTrials))
        print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
        print("Trial Time: " + str(round(time.time()-start_time, 3)) + " sec")
        if totalTrials != 0:
            print("Total Time: " + str(round((time.time()-totalTime_start)/60, 3)) + " min")
        print("Q-Table:")
        for i in range(len(qTable_1)):
            print(str(i) + ": " + str(qTable_1[i]))
        print("change in Q: " + str(changeInQ))
        
    duration = 0
    if converged:
        duration = end_time - start_time
        print(str(round(duration, 3)) + " seconds to converge")
    else:
        duration = None
        print("No convergence")
        
    return (currentEpisode-1), duration

In [48]:
qLearning(2000,1)

Episode: 1/2000
Trial Time: 10.832 sec
Q-Table:
0: [0, -0.99999999999994, -0.99999999999994, 0]
1: [0, -0.9999999999960033, -1.9990958720306642, -0.9999999999960033]
2: [0, -0.9999999997336646, -1.972426015625, -0.9999999997336646]
3: [0, -0.9999999822517003, -1.8067562499999998, -0.9999999492905722]
4: [0, -0.9999995860454878, -1.8585125, -0.9999988172728221]
5: [0, -0.9997748124609375, -1.6168790625, -0.9999211843613282]
6: [0, -0.98499375, -1.5274999999999999, -0.998161734375]
7: [0, -0.8775, -1.3796249999999999, -0.8775]
8: [0, -0.8775, -0.65, -0.65]
9: [0, -0.65, -0.65, -0.65]
10: [0, 0, 0, -0.65]
11: [0, 0, 0, 0]
12: [-0.9999999999999974, -1.9980074410390478, -0.9999999999999974, 0]
13: [-0.9999211843613282, -1.838038921875, -1.9729706650548542, -0.9999211843613282]
14: [-0.998161734375, -1.6168790625, -1.3796249999999999, -1.6656209375]
15: [-0.957125, -1.5274999999999999, -1.55536875, -1.72713125]
16: [-0.9947478125, -1.3796249999999999, -1.4996312500000002, -1.447875]
17: [-0.

KeyboardInterrupt: 